In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2

import pandas as pd

In [2]:
df = pd.read_csv('../data/doctables/all_relevance.csv')
df = df.query('upper_pred>0.5')
print(df.shape)
df.head()

(82368, 12)


,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,mean_prediction,std_prediction,lower_pred,upper_pred
1,1455384,Wetlands occur where biotic and abiotic condit...,High altitude montane wetland vegetation class...,Platberg; Inselberg; Phytosociology; Hydrophyt...,['Plant Sciences'],NaN,0.0,0.0,0.500880,0.060631,0.440249,0.561511
2,1340101,The atmospheric forcing on the Barents Sea ice...,Atmospheric forcing on the Barents Sea winter ...,NaN,['Meteorology & Atmospheric Sciences'],NaN,0.0,0.0,0.554249,0.065086,0.489163,0.619335
12,222401,Sea-level rise and frequent intense hurricanes...,Purple Pitcher Plant (Sarracenia rosea) Diebac...,NaN,['Multidisciplinary Sciences'],NaN,0.0,0.0,0.588284,0.050803,0.537481,0.639087
13,689500,This study analyses long-term water quality da...,Long-term changes in hydrological pathways in ...,DOC; peat; climate change,"['Engineering, Civil; Geosciences, Multidiscip...",NaN,0.0,0.0,0.896791,0.015066,0.881725,0.911857
21,3307272,Rationale: There is significant evidence of in...,Increase in Pediatric Respiratory Visits Assoc...,cough; wheeze; children; wildfire; smoke,['Respiratory System'],NaN,0.0,0.0,0.542398,0.033320,0.509078,0.575717


In [4]:
done = pd.read_csv('../data/doc_continent.csv')

df = df[~df['id'].isin(done['id'])]
df.shape

(13884, 12)

In [6]:
from mordecai import Geoparser
geo = Geoparser()

geo.geoparse("I took the tube from Oxford Circus to London Bridge, via Bank")

[{'country_conf': 0.611725,
  'country_predicted': 'GBR',
  'geo': {'admin1': 'England',
   'country_code3': 'GBR',
   'feature_class': 'S',
   'feature_code': 'BDG',
   'geonameid': '6619889',
   'lat': '51.50821',
   'lon': '-0.08763',
   'place_name': 'London Bridge'},
  'spans': [{'end': 51, 'start': 38}],
  'word': 'London Bridge'}]

In [7]:
places = []
geos = []

for i, row in df.iterrows():
    
    t = row['title'] + row['content']
    t = t.split("Copyright (C)")[0] 
    t = re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",t)[0] 
    t = t.split("Published by Elsevier")[0] 
    t = t.split("Copyright. (C)")[0] 
    t = re.split("\. \(C\) [1-2][0-9]{3} ",t)[0] 
    t = re.split("\. \(C\) Copyright",t)[0]     
    
    gp = geo.geoparse(t)
    
    rplaces = []
    continent = None
    for p in gp:
        try:
            a2 = country_alpha3_to_country_alpha2(p["country_predicted"])
            continent = country_alpha2_to_continent_code(a2)
        except:
            pass
        if "geo" in p:
            try:
                a2 = country_alpha3_to_country_alpha2(p["geo"]["country_code3"])
                continent = country_alpha2_to_continent_code(a2)
            except:
                pass
            p["geo"]["doc_id"] = row['id']
            p["geo"]["word"] = p["word"]
            p["geo"]["country_predicted"] = p["country_predicted"]
            p["geo"]["country_conf"] = p["country_conf"]
            geos.append(p["geo"])
        rplaces.append(p)
    df.loc[i,"continent"] = continent
    df.loc[i,"places"] = len(rplaces)

In [8]:
geo_df = pd.DataFrame.from_dict(geos)
geo_df.head()



,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id
0,Zhejiang,29.87819,121.54945,CHN,1799397,Ningbo,P,PPLA2,1859208
1,NA,30.59822,121.08953,CHN,10098466,Sea View Hangzhou Bay,S,HTL,1859208
2,Zhejiang,29.16667,120,CHN,1784764,Zhejiang Sheng,A,ADM1,1859208
3,Zhejiang,29.16667,120,CHN,1784764,Zhejiang Sheng,A,ADM1,1859208
4,NA,62,10,NOR,3144096,Kingdom of Norway,A,PCLI,3323085


In [9]:
old_geo_df = pd.read_csv('../data/geo_df.csv')
print(old_geo_df.shape)
new_geo_df = pd.concat([old_geo_df,geo_df])
new_geo_df.to_csv('../data/new_geo_df.csv', index=False)

(164006, 9)


In [10]:
print(new_geo_df.shape)

(194919, 9)


In [11]:
place_df = pd.DataFrame.from_dict(places)
place_df.head()


,word,spans,country_predicted,country_conf,geo,doc_id
0,Ningbo,"[{'start': 327, 'end': 333}]",CHN,0.988199,"{'admin1': 'Zhejiang', 'lat': '29.87819', 'lon...",1859208
1,Hangzhou Bay,"[{'start': 432, 'end': 444}]",CHN,0.988199,"{'admin1': 'NA', 'lat': '30.59822', 'lon': '12...",1859208
2,Zhejiang,"[{'start': 446, 'end': 454}]",CHN,0.988199,"{'admin1': 'Zhejiang', 'lat': '29.16667', 'lon...",1859208
3,Zhejiang,"[{'start': 1146, 'end': 1154}]",CHN,0.988199,"{'admin1': 'Zhejiang', 'lat': '29.16667', 'lon...",1859208
4,Norway,"[{'start': 97, 'end': 103}]",NOR,0.999811,"{'admin1': 'NA', 'lat': '62', 'lon': '10', 'co...",3323085


In [12]:
old_place_df = pd.read_csv('../data/place_df.csv')
print(old_place_df.shape)
new_place_df = pd.concat([old_place_df,place_df])
new_place_df.to_csv('../data/new_place_df.csv', index=False)

(286355, 7)


In [13]:
df[['id','continent']].to_csv('../data/doc_continent.csv',index=False)